In [ ]:
pip install googletrans

In [88]:
import pandas as pd
import googletrans
from googletrans import Translator
import json

In [89]:
def remove_word(query):
    stopwords = ['multiple:', 'one:']
    f = lambda x: x.split()
    querywords = query.apply(f)
    f2 = lambda querywords:[word for word in querywords if word.lower() not in stopwords]
    resultwords  = querywords.apply(f2)
    f3 = lambda x : ' '.join(x)
    result = resultwords.apply(f3)
    return result

In [90]:
def back_trans(column):
    #print(column.head())
    translator = Translator()
    ret_text = lambda x: x.text
    trans = lambda x: translator.translate(x, dest='fa').text
    back_trans = lambda x: translator.translate(x, dest='en').text
    trans_col = column.apply(trans)
    #print(trans_col.head())
    #trans_col = trans_col.apply(ret_text)
    backtrans_col = trans_col.apply(back_trans)
    #print(backtrans_col.head())
    return backtrans_col

In [91]:
def create_data(path_q, path_ans, start):
    df = pd.read_csv(path_q, sep='\t',header=None)
    ans = pd.read_csv(path_ans, sep='\t',header=None)

    #select only useful columns, add answers, filter our why questions
    del df[0]
    del df[1]
    df.insert(6, 'ans1', ans.iloc[:,0])
    df.insert(12, 'ans2', ans.iloc[:,1])
    df.insert(18, 'ans3', ans.iloc[:,2])
    df.insert(24, 'ans4', ans.iloc[:,3])
    df[[3, 8, 13, 18]] = df[[3, 8, 13, 18]].apply(remove_word)
    
    #prepare the data
    df_context_before = df.loc[:,[2]]
    df_context = pd.concat([df_context_before, df_context_before, df_context_before, df_context_before])
    df_context = df_context.rename(columns = {2:0})
    df_Q1 = df.loc[:, [3,4,5,6,7,'ans1']]
    df_Q1 = df_Q1.rename(columns = {3:1, 4:2, 5:3, 6:4, 7:5, 'ans1':6})
    df_Q2 = df.loc[:, [8,9,10,11,12,'ans2']]
    df_Q2 = df_Q2.rename(columns = {8:1, 9:2, 10:3, 11:4, 12:5, 'ans2':6})
    df_Q3 = df.loc[:, [13,14,15,16,17,'ans3']]
    df_Q3 = df_Q3.rename(columns = {13:1, 14:2, 15:3, 16:4, 17:5, 'ans3':6})
    df_Q4 = df.loc[:, [18,19,20,21,22,'ans4']]
    df_Q4 = df_Q4.rename(columns = {18:1, 19:2, 20:3, 21:4, 22:5, 'ans4':6})
    frames = [df_Q1, df_Q2, df_Q3, df_Q4]
    df_Qs =  pd.concat(frames)
    df_res = pd.concat([df_context, df_Qs], axis = 1)
    df_res = df_res.reset_index(drop=True)
    #filter why questions
    f_start = lambda x: x.startswith("Why")
    df_res = df_res[df_res[1].apply(f_start)]
    df_res = df_res.reset_index(drop=True)
    
    # back translate the context
    df_res[0] = back_trans(df_res[0])
    # arrange in the format and save into a Json file

    dataset = {}

    for i in range(df_res.shape[0]):
        data = {}
        data["Context"] = df_res.iloc[i,0]
        data["Question"] = df_res.iloc[i,1]
        data["Reasoning type"] = "Causality"
        choices = {}
        choices["A"] = df_res.iloc[i,2]
        choices["B"] = df_res.iloc[i,3]
        choices["C"] = df_res.iloc[i,4]
        choices["D"] = df_res.iloc[i,5]
        data["Choices"] = choices
        data["Answer"] = df_res.iloc[i,6]
        dataset[i + start + 1] = data

    return dataset
    

# Code for MC-Test data

In [92]:
dataset1 = create_data('./MCTest/mc500.train.tsv', './MCTest/mc500.train.ans', 0)

In [93]:
dataset1

{1: {'Context': "The road to my grandfather's house was long and winding. There were many tall trees and animals in the forest. It took a long time because Jimmy was walking in his new boots and he was not used to them yet. He could ride to his grandfather's house, but it would not be fun. Jimmy liked to collect insects on the way to his grandfather's house, so he chose a long route. \\ newline \\ newline As he progressed, Jimmy found more insects to add to his glass. He was interested in electrical faults and found five of them. He also liked butterflies, but they were difficult to catch and took up a lot of space. \\ newline \\ newline Finally, Jimmy arrived at his grandfather's house and knocked. Grandpa answered the door with a smile and greeted Jimmy inside. They sat by the fire and talked about insects. As night fell, they observed electrical faults.",
  'Question': 'Why did Grandpa answer the door?',
  'Reasoning type': 'Causality',
  'Choices': {'A': 'Because he saw the insects

In [94]:
dataset2 = create_data('./MCTest/mc500.dev.tsv', './MCTest/mc500.dev.ans', 150)

In [95]:
dataset2

{151: {'Context': "There lived a stinking elf. He was really mean to everyone around him. He was walking towards the girls. He had put the soap in the refrigerator. She gave shorts as a birthday present. No one likes jinn. One day he was planning a way to make someone sad. He thought of painting on someone's head. He went to someone who was asleep so he could use them. He found a boy lying under a tree. He walked towards her with his magic wand. He painted a giant house on the boy's forehead. The boy woke up and drove the demon away. The boy knew that the jinn had done something wrong to him, so he ran all over the house without looking back. The elf felt really good about himself. She had made the boy sad! He jumped and swam in a nearby pool. The boy never spoke to the jinn again. The elf did not care and continued to harass people.",
  'Question': 'Why did the elf go swimming in a pond?',
  'Reasoning type': 'Causality',
  'Choices': {'A': 'He put soap in the fridge.',
   'B': 'He wa

In [96]:
dataset3 = create_data('./MCTest/mc500.test.tsv', './MCTest/mc500.test.ans', 175)

In [97]:
dataset3

{176: {'Context': 'A little boy named Justin lived on a farm. His father, James, owned the farm. His uncles Jerry, Todd and Bailey also worked on the farm. \\ newline \\ newline The farm had chickens, cows, horses, sheep and pigs. He loved to play with all the farm animals, but horses were his favorite. He rode horses all day. He loved the birth of baby horses. They were born in the spring. When the baby horse is born, Justin\'s father takes the baby out of the barn to make Justin a pet. \\ newline \\ newlineJustin also loved to play on the farm. He was trying to run butterflies out of the meadows. He also liked to stay up late and catch fireflies in the dark. \\ newline \\ newline Justin also loved fishing in the summer. On Sunday evening Tuesday, he went to the farm pond and caught a big catfish. This fish was so big that it almost broke his fishing pole. Justin brought in the fish. \\ newline \\ newline The fish looked at Justin with sad eyes. Justin felt bad seeing the fish he had 

In [98]:
dataset4 = create_data('./MCTest/mc160.train.tsv', './MCTest/mc500.train.ans', 236)

In [99]:
dataset4

{237: {'Context': 'Two best friends lived in a small town. They grew up together and did everything together. There was a boy named James and he was a cute and fat kid who was always chosen \\ newlineon. And then another boy named Alex, and he was handsome and cool. They were \\ newlineopposites, but they did not care. Alex always protected James from bullies because \\ newlineJames was weak. James did not know what to do and was scared. But then he remembered all the new things that Alex had done to replace him. James looked at Alex and considered him his new hero. James wanted to protect himself and Alex did not always help him. So this time, \\ newlineJames got up and was brave. He stood up to Gordon and Johnny and told them he was not a new caretaker. The bullies did not know what to do and then gave up. They went to look for interesting new things. Alex saw everything and was proud of James for his courage. \\ newline \\ newline Now James and Alex protect each other and stand agai

In [100]:
dataset5 = create_data('./MCTest/mc160.dev.tsv', './MCTest/mc500.dev.ans', 262)

In [101]:
dataset5

{263: {'Context': 'Sarah wanted to play on a baseball team. He has never tried to spin a bat before and attack a new baseball. His father gave him a bat and they went to the park together to \\ newlinepractice. Sarah wondered if she could hit the ball. He was not sure he would ever be \\ newlinegood. She really liked to play in a team and wear a real dress. He could not wait for \\ newlineget to come to the park and test his bat. When Sarah and her father arrived at the park, Sarah grabbed the new bat and stood a few steps away from her father. Sarah was waiting when her father threw the ball. \\ Newlineto. Her heart was pounding. He lost the first few pitches. He felt he was giving up on the new option but was still trying. Soon he was throwing the ball too far. He was very happy and did not wait to join a real team. His father was very proud of him for not giving him a new lineup.',
  'Question': 'Why was Sara practicing?',
  'Reasoning type': 'Causality',
  'Choices': {'A': 'She wan

In [102]:
dataset6 = create_data('./MCTest/mc160.test.tsv', './MCTest/mc500.test.ans', 276)

In [103]:
dataset6

{277: {'Context': "Hannah Harvey was ten years old and had many friends at school. He lived in New York and enjoyed gymnastics and playing football. One day Hannah came home from school and was greeted by her parents. He knew something was wrong with their facial expressions. Even Jackson, Hannah's dog, acted differently. Hannah asked why everyone is so weird? Hannah's father, known as the Pope, explained to Hannah that his work was forcing him to move. Hannah did not seem to think this was a very big deal. Then, Hannah's mother explained that they were moving to Kenya. Kenya, he explained, is a place in Africa and life there will be very different. When Hannah started crying and thinking about all her friends at home, Hannah's mother calmed her down with a gentle touch. Jackson starts howling with Hannah crying, but Hannah's mother calms him down too. Hannah spent the next two weeks visiting her friends and saying goodbye. He did not know the next time he came home. She cried very har

In [104]:
# combine the datasets
final_dataset = {**dataset1, **dataset2, **dataset3, **dataset4, **dataset5, **dataset6}

In [105]:
#Final dataset extracted from MC-Test
final_dataset

{1: {'Context': "The road to my grandfather's house was long and winding. There were many tall trees and animals in the forest. It took a long time because Jimmy was walking in his new boots and he was not used to them yet. He could ride to his grandfather's house, but it would not be fun. Jimmy liked to collect insects on the way to his grandfather's house, so he chose a long route. \\ newline \\ newline As he progressed, Jimmy found more insects to add to his glass. He was interested in electrical faults and found five of them. He also liked butterflies, but they were difficult to catch and took up a lot of space. \\ newline \\ newline Finally, Jimmy arrived at his grandfather's house and knocked. Grandpa answered the door with a smile and greeted Jimmy inside. They sat by the fire and talked about insects. As night fell, they observed electrical faults.",
  'Question': 'Why did Grandpa answer the door?',
  'Reasoning type': 'Causality',
  'Choices': {'A': 'Because he saw the insects

In [106]:
# save into json
with open('Causality_Data_MC.json', 'w') as outfile:
    outfile.write(json.dumps(final_dataset, indent=4))

# Code for Cosmos dataset

In [17]:
#Cosmos QA data
train = pd.read_csv("./data/train.csv")
#train = train.iloc[0:100,:]

In [18]:
train.head()

,id,context,question,answer0,answer1,answer2,answer3,label
0,3Q9SPIIRWJKVQ8244310E8TUS6YWAC##34V1S5K3GTZMDU...,Good Old War and person L : I saw both of thes...,"In the future , will this person go to see oth...",None of the above choices .,This person likes music and likes to see the s...,This person only likes Good Old War and Person...,Other Bands is not on tour and this person can...,1
1,3E24UO25QZOMYXHZN4TEH9EMT9GO6L##3UN61F00HXNWYQ...,I mean it : not one person said ANYTHING to me...,Why might have the temp agency tell me I am no...,The company hiring the temp workers might have...,The temp agency hiring the temp workers might ...,None of the above choices .,I might have had a change of mind working for ...,0
2,3M4KL7H8KVL125AYH2V35D1E0A016T##3SB5N7Y3O426ET...,Leaving my shift Thursday day shift I arrived ...,What may have caused the radio to erupt with d...,My partner needed a medic unit .,Someone was running from the ambulances after ...,None of the above choices .,Someone was running from the cops and got into...,3
3,3D5G8J4N5CI2K40F4RZLF9OG2L4VTJ##34MAJL3QP721EU...,"So , last day in Seattle , and my flight was a...",Why did I chit chat with my old manager ?,Because my flight was at 1:30 .,Because I left Seattle feeling really good and...,Because it 's my last day in Seattle .,Because I enjoy talking to him .,3
4,3MQKOF1EE428I44N8B42W7P86DKDW4##3NJM2BJS4ZLBGN...,Ms. Mumma informed Mr. Dail that evidence had ...,Why did I burst into tears ?,Because Ms. Mumma was informative .,Because I had an intense emotional response to...,Because Mr. Dail was informed .,Because I fell out of my chair .,1


In [19]:
train.shape

(25262, 8)

In [113]:
train__final = []

In [38]:
#Back translate the context for 10k selected data
for i in range(100):
    train_new = train.iloc[100 * i:100 * (i + 1),:]
    print(train_new.shape)
    backtrans=back_trans(train_new['context'])
    train_data.append(backtrans)


(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)
(100, 8)


In [77]:
final_context = pd.concat(train_final)

In [78]:
final_context.shape

(10000,)

In [109]:
#put in json format
dataset = {}
start = 313
for i in range(final_context.shape[0]):
        data = {}
        data["Context"] = final_context[i]
        data["Question"] = train.iloc[i,2]
        data["Reasoning type"] = "Causality"
        choices = {}
        choices["A"] = train.iloc[i,3]
        choices["B"] = train.iloc[i,4]
        choices["C"] = train.iloc[i,5]
        choices["D"] = train.iloc[i,6]
        data["Choices"] = choices
        data["Answer"] = chr(65 + train.iloc[i,7])
        dataset[i + start + 1] = data

In [111]:
# save into json
with open('Causality_Data_Cosmos.json', 'w') as outfile:
    outfile.write(json.dumps(dataset, indent=4))

# Combine Mc-Test and Cosmos data

In [112]:
# Combine Cosmos and MC-Test
combined_dataset = {**final_dataset, **dataset}
# save into json
with open('Causality_Data_Final.json', 'w') as outfile:
    outfile.write(json.dumps(combined_dataset, indent=4))